In [1]:
import pandas as pd
import numpy as np
from dgutils.pandas import add_columns

/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/genome_kit/data_manager.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
df_dataset = pd.read_pickle('/Users/alicegao/work/psi-lab-sandbox/rna_ss/data_processing/rna_cg/data/s_processed.pkl')
df_e2efold = pd.read_pickle('/Users/alicegao/work/psi-lab-sandbox/rna_ss/tools/e2efold_2/e2efold_productive/tmp/s_processed.pkl')
df_spotrna = pd.read_pickle('/Users/alicegao/work/psi-lab-sandbox/rna_ss/tools/SPOT-RNA-master/tmp/s_processed.pkl')
df_rnafold = pd.read_pickle('/Users/alicegao/work/psi-lab-sandbox/meetings/2020_05_26/data/s_processed.pkl')

In [3]:
df_e2efold = df_e2efold.rename(columns={'pred_idx': 'pred_idx_e2efold'})
df = pd.merge(df_dataset, df_e2efold[['seq', 'pred_idx_e2efold']], on='seq')
df_spotrna = df_spotrna.rename(columns={'pred_idx': 'pred_idx_spotrna'})
df = pd.merge(df, df_spotrna[['seq', 'pred_idx_spotrna']], on='seq')
df_rnafold = df_rnafold.rename(columns={'pred_idx': 'pred_idx_rnafold'})
df = pd.merge(df, df_rnafold[['seq', 'pred_idx_rnafold']], on='seq')

In [11]:
# load dataset overlap, so that we can check performance on held-out portion

# SPOt-RNA was training on bpRNA & PDB20
df_overlap_spotrna1 = pd.read_csv('/Users/alicegao/work/psi-lab-sandbox/rna_ss/data_processing/dataset_overlap/data/sprocessed_bprna_overlap.csv.gz')
df_overlap_spotrna2 = pd.read_csv('/Users/alicegao/work/psi-lab-sandbox/rna_ss/data_processing/dataset_overlap/data/sprocessed_pdb250_overlap.csv.gz')
df_overlap_spotrna = pd.concat([df_overlap_spotrna1, df_overlap_spotrna2])

# E2Efold trained on rnastralign
df_overlap_e2efold = pd.read_csv('/Users/alicegao/work/psi-lab-sandbox/rna_ss/data_processing/dataset_overlap/data/sprocessed_rnastralign_overlap.csv.gz')


In [12]:
ids_in_spotrna = df_overlap_spotrna['seq_id1'].unique()
ids_in_e2efold = df_overlap_e2efold['seq_id1'].unique()


print(len(ids_in_spotrna))
print(len(ids_in_e2efold))

810
1081


In [4]:
class EvalMetric(object):

#     @staticmethod
#     def _check_arr(arr):
#         assert len(arr.shape) == 2
#         assert arr.shape[0] == arr.shape[1]
#         assert np.all((arr == 0) | (arr == 1))
#         assert np.max(np.sum(arr, axis=0)) <= 1
#         assert np.max(np.sum(arr, axis=1)) <= 1

    @staticmethod
    def sensitivity(_pred, _target):
        # numerator: number of correct predicted base pairs
        # denominator: number of true base pairs
        assert _pred.shape[0] == _target.shape[0]
        n = _pred.shape[0]
        # set lower triangular to be all 0's
        pred = _pred.copy()
        target = _target.copy()
        pred[np.tril_indices(n)] = 0
        target[np.tril_indices(n)] = 0
        # checks
#         EvalMetric._check_arr(pred)
#         EvalMetric._check_arr(target)
        # metric
        idx_true_base_pair = np.where(target == 1)
        return float(np.sum(pred[idx_true_base_pair]))/np.sum(target)

    @staticmethod
    def ppv(_pred, _target):
        # numerator: number of correct predicted base pairs
        # denominator: number of predicted base pairs
        assert _pred.shape[0] == _target.shape[0]
        n = _pred.shape[0]
        # set lower triangular to be all 0's
        pred = _pred.copy()
        target = _target.copy()
        pred[np.tril_indices(n)] = 0
        target[np.tril_indices(n)] = 0
        # checks
#         EvalMetric._check_arr(pred)
#         EvalMetric._check_arr(target)
        # metric
        idx_predicted_base_pair = np.where(pred == 1)
        return float(np.sum(target[idx_predicted_base_pair])/np.sum(pred))

    @staticmethod
    def f_measure(sensitivity, ppv):
        return (2 * sensitivity * ppv)/(sensitivity + ppv)

In [5]:
eval_metric = EvalMetric()

In [6]:
def add_metrics(seq_len, one_idx, pred_idx):
    arr_target = np.zeros((seq_len, seq_len))
    arr_target[one_idx] = 1
    arr_pred = np.zeros((seq_len, seq_len))
    arr_pred[pred_idx] = 1

    sensitivity = eval_metric.sensitivity(arr_pred, arr_target)
    ppv = eval_metric.ppv(arr_pred, arr_target)
    f_measure = eval_metric.f_measure(sensitivity, ppv)
    
    return sensitivity, ppv, f_measure

In [7]:
for model_name in ['e2efold', 'spotrna', 'rnafold']:
    df = add_columns(df, ['sensitivity_{}'.format(model_name),
                          'ppv_{}'.format(model_name),
                          'f_measure_{}'.format(model_name)], 
                     ['len', 'one_idx', 'pred_idx_{}'.format(model_name)], add_metrics)

/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """
/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in double_scalars
/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars


In [8]:
df.describe()

,f_measure_e2efold,f_measure_rnafold,f_measure_spotrna,len,ppv_e2efold,ppv_rnafold,ppv_spotrna,sensitivity_e2efold,sensitivity_rnafold,sensitivity_spotrna
count,4541.000000,4929.000000,5117.000000,5188.000000,5128.000000,5096.000000,5168.000000,5188.000000,5188.000000,5188.000000
mean,0.329789,0.685928,0.680971,170.622205,0.371795,0.645606,0.697781,0.257972,0.677148,0.678636
std,0.303762,0.246408,0.227362,175.235356,0.336756,0.278900,0.202424,0.305469,0.285471,0.289145
min,0.007366,0.019868,0.022472,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.071429,0.489051,0.493671,36.000000,0.068966,0.438272,0.576760,0.029703,0.482759,0.415976
50%,0.222222,0.724138,0.700000,83.000000,0.263523,0.674078,0.714286,0.136603,0.730769,0.750000
75%,0.488889,0.906667,0.888889,316.000000,0.657331,0.897414,0.840000,0.333333,0.933333,1.000000
max,1.000000,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
# subset to sequences not similar/identical to SPOT-RNA training data
df[~df['seq_id'].isin(ids_in_spotrna)].describe()

,f_measure_e2efold,f_measure_rnafold,f_measure_spotrna,len,ppv_e2efold,ppv_rnafold,ppv_spotrna,sensitivity_e2efold,sensitivity_rnafold,sensitivity_spotrna
count,3751.000000,4120.000000,4290.000000,4360.000000,4300.000000,4272.000000,4341.000000,4360.000000,4360.000000,4360.000000
mean,0.259182,0.686147,0.661127,180.672936,0.318681,0.647490,0.691880,0.186681,0.670124,0.643497
std,0.239964,0.250925,0.233905,184.826943,0.306987,0.286205,0.211230,0.227326,0.291448,0.292171
min,0.007366,0.019868,0.022472,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.060606,0.475524,0.463158,30.000000,0.058824,0.429229,0.562500,0.024390,0.466777,0.384615
50%,0.191083,0.721311,0.666667,83.000000,0.214286,0.673691,0.702381,0.111111,0.717787,0.647059
75%,0.387860,0.909091,0.877193,352.250000,0.500000,0.928571,0.848485,0.256544,0.936503,1.000000
max,1.000000,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
# subset to sequences not similar/identical to E2Efold training data
df[~df['seq_id'].isin(ids_in_e2efold)].describe()

,f_measure_e2efold,f_measure_rnafold,f_measure_spotrna,len,ppv_e2efold,ppv_rnafold,ppv_spotrna,sensitivity_e2efold,sensitivity_rnafold,sensitivity_spotrna
count,3270.000000,3657.000000,3820.000000,3881.000000,3827.000000,3801.000000,3865.000000,3881.000000,3881.000000,3881.000000
mean,0.221438,0.698872,0.679034,163.049472,0.280836,0.659375,0.703065,0.151751,0.679267,0.664765
std,0.217226,0.254569,0.238035,177.844713,0.297128,0.292085,0.214325,0.198245,0.297751,0.295811
min,0.007366,0.019868,0.041667,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.053068,0.486322,0.470588,28.000000,0.048387,0.433333,0.569444,0.020000,0.473282,0.393939
50%,0.166667,0.745098,0.714286,74.000000,0.166667,0.698413,0.727273,0.076923,0.737705,0.708333
75%,0.315789,0.933333,0.896552,316.000000,0.444444,0.947368,0.862069,0.214286,1.000000,1.000000
max,1.000000,1.000000,1.000000,600.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
